In [73]:
from binance.client import Client
import pandas as pd
import requests

def get_historical_prices(api_key, api_secret, symbol, start_date, end_date):
    client = Client(api_key, api_secret)
    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start_date, end_date)
    df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignored'])
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df.set_index('open_time', inplace=True)
    df.drop(columns=['ignored'], inplace=True)
    return df


In [74]:
def get_crypto_data(api_key, api_secret, crypto_list, start_date, end_date):
    for symbol in crypto_list:
        try:
            # Retrieve historical prices for the cryptocurrency
            df = get_historical_prices(api_key, api_secret, symbol + 'USDT', start_date, end_date)
            df.to_csv(symbol + '.csv')
        except Exception as e:
            print(f'Failed to retrieve data for {symbol}. Error: {str(e)}')


In [75]:
binance_api_key = ''
binance_secret_key = ''

start_date = '2018-05-01'
end_date = '2023-05-01'

crypto_symbols = ['BTC', 'ETH', 'BNB', 'ADA', 'SOL', 'DOGE']

get_crypto_data(binance_api_key, binance_secret_key, crypto_symbols, start_date, end_date)

In [ ]:
import pytz
import pandas as pd
import numpy as np
import ta

crypto_data = {}

crypto_symbols = ['BTC', 'ETH', 'BNB', 'ADA', 'SOL', 'DOGE']

crypto_data['ADA'] = pd.read_csv('/content/drive/MyDrive/ADA.csv')
crypto_data['BNB'] = pd.read_csv('/content/drive/MyDrive/BNB.csv')
crypto_data['BTC'] = pd.read_csv('/content/drive/MyDrive/BTC.csv')
crypto_data['DOGE'] = pd.read_csv('/content/drive/MyDrive/DOGE.csv')
crypto_data['ETH'] = pd.read_csv('/content/drive/MyDrive/ETH.csv')
crypto_data['SOL'] = pd.read_csv('/content/drive/MyDrive/SOL.csv')

# Step 1: Data Cleaning and Formatting
def clean_data(df):
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ns').dt.floor('ns').dt.tz_localize(pytz.utc)
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ns').dt.floor('ns').dt.tz_localize(pytz.utc)

    df = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df = df.set_index('date')
    df = df.sort_index()
    df = df.fillna(method='ffill')
    return df

for symbol in crypto_symbols:
    df = crypto_data[symbol]
    df = clean_data(df)
    crypto_data[symbol] = df

# Step 2: Technical Indicator Creation
def create_technical_indicators(df):
    df['daily_return'] = df['close'].pct_change()
    df['ma_7'] = ta.volatility.bollinger_mavg(df['close'], window=7)
    df['ma_14'] = ta.volatility.bollinger_mavg(df['close'], window=14)
    df['ma_21'] = ta.volatility.bollinger_mavg(df['close'], window=21)
    df['rsi_7'] = ta.momentum.rsi(df['close'], window=7)
    df['rsi_14'] = ta.momentum.rsi(df['close'], window=14)
    df['rsi_21'] = ta.momentum.rsi(df['close'], window=21)
    df['lower_band'] = ta.volatility.bollinger_lband(df['close'], window=20, window_dev=2)
    df['upper_band'] = ta.volatility.bollinger_hband(df['close'], window=20, window_dev=2)
    df['middle_band'] = ta.volatility.bollinger_mavg(df['close'], window=20)
    df['stoch'] = ta.momentum.stoch(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    macd = ta.trend.macd(df['close'], window_slow=26, window_fast=12)
    signal_line = macd.ewm(span=9, adjust=False).mean()
    histogram = macd - signal_line

    df['macd'] = macd
    df['signal_line'] = signal_line
    df['histogram'] = histogram

    return df

for symbol in crypto_symbols:
    df = crypto_data[symbol]
    df = create_technical_indicators(df)
    crypto_data[symbol] = df

# Step 3: Feature Engineering
def create_features(df):
    df['open_trend'] = df['open'] - df['close'].shift(1)
    df['close_trend'] = df['close'] - df['close'].shift(1)
    df['high_trend'] = df['high'] - df['high'].shift(1)
    df['low_trend'] = df['low'] - df['low'].shift(1)
    df['volume_trend'] = df['volume'] - df['volume'].shift(1)
    df['return_trend'] = df['daily_return'] - df['daily_return'].shift(1)
    df = df.dropna()
    return df


for symbol in crypto_symbols:
    df = crypto_data[symbol]
    df = create_features(df)
    crypto_data[symbol] = df


In [85]:
crypto_data['BTC']

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql import SparkSession
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

# create a PySpark session
spark = SparkSession.builder.appName('CryptoReturn').getOrCreate()

# set the list of cryptocurrencies to predict
crypto_symbols = ['BTC', 'ETH', 'BNB', 'ADA', 'SOL', 'DOGE']

# create a list to hold the trained models
models = []

# loop through each cryptocurrency
for symbol in crypto_symbols:
    # get the cleaned and feature engineered data for this cryptocurrency
    df = crypto_data[symbol]

    # define the feature columns
    feature_cols = ['open', 'high', 'low', 'close', 'volume', 'ma_7', 'ma_14', 'ma_21', 'rsi_7', 'rsi_14', 'rsi_21', 'lower_band', 'upper_band', 'middle_band', 'stoch', 'macd', 'signal_line', 'histogram', 'open_trend', 'close_trend', 'high_trend', 'low_trend', 'volume_trend', 'return_trend']

    # create a PySpark DataFrame from the data
    spark_df = spark.createDataFrame(df.reset_index())

    # create a VectorAssembler to combine the feature columns into a single vector column
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

    # scale the features
    scaler = MinMaxScaler(inputCol='features', outputCol='scaled_features')

    # split the data into training and testing sets
    training_data, testing_data = spark_df.randomSplit([0.8, 0.2], seed=42)

    # create a GBTRegressor model
    gbt = GBTRegressor(featuresCol='scaled_features', labelCol='daily_return', maxDepth=5, maxBins=32, maxIter=20)

    # create a pipeline to run the feature engineering and model training steps
    pipeline = Pipeline(stages=[assembler, scaler, gbt])

    # create a grid of hyperparameters to search over
    param_grid = ParamGridBuilder() \
        .addGrid(gbt.maxDepth, [5, 10]) \
        .addGrid(gbt.maxBins, [32, 64]) \
        .build()

    # create a cross validator to search over the hyperparameter grid and evaluate the model performance
    evaluator = RegressionEvaluator(metricName='rmse', labelCol='daily_return')
    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

    # train the model using the training data and the cross validator
    cv_model = cv.fit(training_data)

    # evaluate the model on the testing data
    predictions = cv_model.transform(testing_data)
    rmse = evaluator.evaluate(predictions)
    accuracy = 1 - rmse
    # print the accuracy for the model
    print(f'{symbol}: Accuracy = {accuracy}')

    # add the trained model to the list of models
    models.append((symbol, cv_model))
